* Apply to autism variants from zhou et al., 2019 https://hb.flatironinstitute.org/asdbrowser/

* The tsv file was converted to vcf format

## Prepare data as VCF file

In [1]:
import mmsplice
mmsplice.__version__

Using TensorFlow backend.
/opt/modules/i12g/anaconda/3-5.0.1/envs/mtsplice_paper/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/modules/i12g/anaconda/3-5.0.1/envs/mtsplice_paper/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/modules/i12g/anaconda/3-5.0.1/envs/mtsplice_paper/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint1

'1.0.1'

In [2]:
from mtsplice import get_data_dir
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from mtsplice.utils.vcf_utils import variant_name_convert

DATA = get_data_dir()

## Apply MMSplice first

In [3]:
from mmsplice.vcf_dataloader import SplicingVCFDataloader
from mmsplice import MMSplice, predict_all_table
from mmsplice.utils import max_varEff

In [4]:
gtf = '/data/nasif12/home_if12/chengju/project/recods/Data/shared/Homo_sapiens.GRCh37.75.uniq_exon.gtf'
fasta = '/data/nasif12/home_if12/chengju/project/recods/Data/shared/hg19.nochr.fa'
vcf = DATA + "dis_asd.vcf"

In [5]:
dl = SplicingVCFDataloader(gtf, 
                          fasta,
                          vcf,
                          split_seq=True, 
                          overhang=(300, 300))

In [6]:
model = MMSplice()

Instructions for updating:
Colocations handled automatically by placer.


2020-05-24 23:22:02,776 [WARNING] From /opt/modules/i12g/anaconda/3-5.0.1/envs/mtsplice_paper/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


2020-05-24 23:22:03,258 [WARNING] From /opt/modules/i12g/anaconda/3-5.0.1/envs/mtsplice_paper/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
import warnings
warnings.filterwarnings('ignore')

predictions = predict_all_table(model, dl, batch_size=1024)

23it [17:29, 46.67s/it]


In [8]:
predictions.shape

(23537, 17)

In [9]:
predictionsMax = max_varEff(predictions)

In [10]:
predictionsMax["ID"] = predictionsMax.ID.apply(variant_name_convert)

In [11]:
autism = pd.read_csv(f"{DATA}/dis_asd.tsv.gz", comment='#', sep='\t', index_col=None)
autism['ID'] = autism.Chr.map(str) + '_' + autism.Pos.map(str) + '_' + autism.Ref.map(str) + '_' + autism.Alt.map(str)

In [12]:
pred = pd.merge(predictionsMax, autism, on=['ID'])

## Intesect with Observed over Expected score

In [13]:
oe = pd.read_csv(f"{DATA}/oe_clean.tsv", sep='\t')

In [14]:
pred_oe = pd.merge(pred, oe, left_on="RNA gene symbol", right_on="gene")

In [15]:
pred_oe['lof_gene'] = pred_oe.oe_lof_upper < 0.35

In [16]:
pred_oe.to_csv(DATA + "MMSplice_pred_oe.csv", index=None)